# Pandas Tricks

*Rappel et/ou découverte de manipulation de matrices de données avec pandas*

### Lire un tableur



In [25]:
import pandas as pd

# préciser le chemin du fichier à lire
cheminDuFichierCSV = "./../data/dictionary/1757_Buchanan-J.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")

# afficher à l'écran un échantillon du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
7340,3,invīʹr_o_n,F.,NaN,NaN,v.,30.0,123
12902,3,ſpĭʹſsĭtūde,L.,NaN,NaN,n.,8.0,201
4139,1,dūke,F.,NaN,NaN,n.,15.0,77
2854,1,coom,NaN,NaN,NaN,n.,19.0,59
1631,2,bŭʹllet,NaN,NaN,NaN,n.,14.0,42


Pour lires des tableurs aux format xls, xlsx ou odt il faut utiliser pd.read_excel(cheminDuFichier) et je conseil de bien préciser explicitement l'engine à utiliser en paramètre.
* dfXlsx = pd.read_excel(cheminDuFichierXLSX, engine="openpyxl")
* dfOds = pd.read_excel(cheminDuFichierODS, engine="odf")

### Ajouter un nouvelle colonne contenant le resultat d'un traitement

In [26]:
# définir une fonction qui retourne quelque chose
def sizePron(strPron):
    #retourner le nombre de caractère de la prononciation
    
    return len(strPron)



In [27]:
# appeler cette fonction sur toutes les lignes de la matrice pour remplir une nouvelle colonne grace à apply
dfDicoBuch['sizePron'] = dfDicoBuch.apply(lambda x: sizePron(x['pron']),1)

dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
10310,1,pŏd,NaN,NaN,NaN,n.,69.0,164,3
15761,1,yĕrk,NaN,NaN,NaN,v.,7.0,241,4
732,2,ărraiʹgn,F.,NaN,NaN,v.,50.0,29,8
14318,3,tympāʹnum,NaN,NaN,NaN,n.,59.0,220,9
12323,3,ſeiʹ_g_nĭor,NaN,NaN,NaN,n.,6.0,193,11


## Un petit calcul

In [28]:
dfDicoBuch['sizePron'].median()

9.0

## Une recherche de valeur

In [29]:
dfDicoBuch[dfDicoBuch['sizePron']==26.0]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
2813,3,contrŏʹller or comptrollĕr,NaN,NaN,NaN,n.,49.0,58,26
7492,3,kaʹlĕnd_a_r or caʹlĕnd_a_r,NaN,NaN,NaN,n.,2.0,125,26


## Manipuler des matrices de format exotique

In [33]:
cheminDuDicoBigi = "./../data/dictionary/2015_Bigi-B.dict" 

# ouvrir une matric exotique
#dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" [] ", encoding="utf8")

dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" \[\] ", encoding="utf8", engine='python', names=['hwd','Sampa'], skiprows=8 )

# afficher à l'écran un échantillon du tableau
dfDicoBigi.head(n=5)

,hwd,Sampa
0,a,@
1,a(2),eI
2,a'ready,@ r\ E d i:
3,a's,eI z
4,a.,eI


Intro/rappel Regex : https://gitlab.huma-num.fr/mnauge/cnam-athon-transcrire/-/blob/master/codes/regexBasics.ipynb

## Jouer avec deux matrices à la fois

In [34]:
cheminDuMappingSampaIpa = "./../data/dictionary/sampa-ipa.txt" 
dfMappingSampaIpa = pd.read_csv(cheminDuMappingSampaIpa, delimiter="  *", encoding="utf8", engine='python', names=['sampa','ipa'])
dfMappingSampaIpa.sample(n=3)

,sampa,ipa
56,M,ɯ
48,I:,ɪː
117,_x,̽


In [35]:
dfMappingSampaIpa['sampa'].value_counts()

r      3
h      2
g      2
U~/    2
@      2
      ..
O:     1
P      1
Q      1
R      1
~      1
Name: sampa, Length: 177, dtype: int64

In [36]:
dfMappingSampaIpa[dfMappingSampaIpa['sampa']=='@']

,sampa,ipa
28,@,ə
29,@,ɛ


In [37]:
# ne garder que la première occurence pour éviter une indécision mais ce n'est pas terrible. A discuter avec les linguistes.
dfMappingSampaIpa.drop_duplicates(subset=['sampa'], keep='first', inplace=True)

In [38]:
dfMappingSampaIpa[dfMappingSampaIpa['sampa']=='@']

,sampa,ipa
28,@,ə


In [46]:
def sampa2ipa(strSampa, dfMapping):
    ipaRes = ""
    
    listValSampa = strSampa.split(" ")
    #print(listValSampa)
    for valSampa in listValSampa:
        dfRowFinded = dfMapping[dfMapping['sampa']==valSampa]

        if len(dfRowFinded==1):
            ipaRes+=dfRowFinded.iloc[0]['ipa']
        else:
            ipaRes+='?'
    return ipaRes

In [40]:
sampa2ipa("A r\ t i",dfMappingSampaIpa)

'ɑɹti'

In [48]:
dfDicoBigiRich = dfDicoBigi.copy()

dfDicoBigiRich['ipa'] = dfDicoBigiRich.apply(lambda x: sampa2ipa(x['Sampa'], dfMappingSampaIpa),1)


In [49]:
dfDicoBigiRich.sample(n=5)

,hwd,Sampa,ipa
84295,opponent,@ p @U n @ n t,əpoʊnənt
95531,readjusting,r\ i: @ dZ V s t i N,ɹiːə?ʌstiŋ
109939,sordoni,s O: r\ d @U n i:,sOːɹdoʊniː
108384,skinnier,s k I n i: 3:r,skɪniːɜ:r
121832,unilaterally,j u n @ l { 4 3:r @ l i:,junəlæɾɜ:rəliː


In [50]:
dfDicoBigiRich

,hwd,Sampa,ipa
0,a,@,ə
1,a(2),eI,?
2,a'ready,@ r\ E d i:,əɹɛdiː
3,a's,eI z,?z
4,a.,eI,?
...,...,...,...
131098,zyuganov's(3),z u g A n A v z,zugɑnɑvz
131099,zyuganov's(4),z u g A n A f s,zugɑnɑfs
131100,zywicki,z i w I k i:,ziwɪkiː
131101,zzzz,z i: z,ziːz


### Sauvegarder la nouvelle matrice


In [52]:
pathDicoBigiRich = "./../data/dataframe/BigiRichSample.csv"
dfDicoBigiRich.to_csv(pathDicoBigiRich, sep=";",encoding="utf8",index=False)